In [142]:
import pandas as pd
from pathlib import Path
import statsmodels.formula.api as sm
import numpy as np

In [143]:
interim = '../data/interim'
br = pd.read_pickle(Path(interim) / 'batting_records.pkl')
gl = pd.read_pickle(Path(interim) / 'game_logs.pkl')

In [144]:
br = br.sort_values(['BAT_ID', 'year'])
br_pred = br.loc[:, ['BAT_HAND', 'G', 'HR', 'PA']] # .groupby('BAT_ID').shift(1)
br_pred = br_pred.dropna(subset=['G'])

In [212]:
def marcel2(ser):
    temp = harp.loc[ser.index]
    
    if len(temp) == 1:
        hr = temp['HR'][0]*5
        pa = temp['PA'][0]*.5 + 200
    elif len(temp) == 2:
        hr = temp['HR'][1]*5 + temp['HR'][0]*4
        pa = temp['PA'][1]*.5 + temp['PA'][0]*.1 + 200
    else:
        hr = temp['HR'][2]*5 + temp['HR'][1]*4 + temp['HR'][0]*3
        pa = temp['PA'][2]*.5 + temp['PA'][1]*.1 + 200
       
    y = temp.index.get_level_values('year')[0]
    print([y, hr, pa])
    return hr

In [213]:
harp = br_pred[br_pred.index.get_level_values('BAT_ID') == 'harpb003']

In [175]:
print(harp.groupby('BAT_ID')['HR'].rolling(3, 1).apply(marcel2, raw=False).to_string())

[2012, 110, 497.0]
[2012, 188, 506.4]
[2012, 211, 445.4]
[2013, 322, 566.2]
[2014, 327, 578.9]
[2015, 367, 508.7]
[2016, 358, 596.7]
[2017, 398, 610.5]
[2018, 307, 389.7]
BAT_ID  
harpb003    110.0
harpb003    188.0
harpb003    211.0
harpb003    322.0
harpb003    327.0
harpb003    367.0
harpb003    358.0
harpb003    398.0
harpb003    307.0


In [191]:
temp = gl[['GAME_ID', 'year', 'HomeTeamLeague', 'VisitingTeamLeague', 'HomeHR', 'VisitorHR']]

temp.columns = ['GAME_ID', 'year', 'LeagueHome', 'LeagueVisitor', 'HRHome', 'HRVisitor']

b = pd.wide_to_long(temp, ['League', 'HR'], i=['GAME_ID', 'year'], j='Type', suffix=r'\w+')

LeagueHR = b.groupby(['League', 'year'])[['HR']].sum()
LeagueHR = LeagueHR[LeagueHR.index.get_level_values('League').isin(['AL','NL'])]